In [1]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_core.documents import Document
from langchain_chroma import Chroma
from sqlalchemy import create_engine
from transformers import AutoModel, AutoTokenizer, pipeline
import pandas as pd

In [3]:
engine = create_engine("sqlite:////home/karysoares/Documents/book-reviews/notebooks/books.db")

In [4]:
books_data = pd.read_sql('SELECT * FROM books_data', engine)

In [5]:
books_data.head()

,Title,description,authors,image,previewLink,publisher,publishedDate,infoLink,categories,ratingsCount,review_mean,review_ids
0,Its Only Art If Its Well Hung!,None,['Julie Strain'],http://books.google.com/books/content?id=DykPA...,http://books.google.nl/books?id=DykPAAAACAAJ&d...,None,1996,http://books.google.nl/books?id=DykPAAAACAAJ&d...,['Comics & Graphic Novels'],NaN,4.000000,['1882931173']
1,Dr. Seuss: American Icon,Philip Nel takes a fascinating look into the k...,['Philip Nel'],http://books.google.com/books/content?id=IjvHQ...,http://books.google.nl/books?id=IjvHQsCn_pgC&p...,A&C Black,2005-01-01,http://books.google.nl/books?id=IjvHQsCn_pgC&d...,['Biography & Autobiography'],NaN,4.555556,"['0826414346', '0826414346', '0826414346', '08..."
2,Wonderful Worship in Smaller Churches,This resource includes twelve principles in un...,['David R. Ray'],http://books.google.com/books/content?id=2tsDA...,http://books.google.nl/books?id=2tsDAAAACAAJ&d...,None,2000,http://books.google.nl/books?id=2tsDAAAACAAJ&d...,['Religion'],NaN,5.000000,"['0829814000', '0829814000', '0829814000', '08..."
3,Whispers of the Wicked Saints,Julia Thomas finds her life spinning out of co...,['Veronica Haddon'],http://books.google.com/books/content?id=aRSIg...,http://books.google.nl/books?id=aRSIgJlq6JwC&d...,iUniverse,2005-02,http://books.google.nl/books?id=aRSIgJlq6JwC&d...,['Fiction'],NaN,3.718750,"['0595344550', '0595344550', '0595344550', '05..."
4,"Nation Dance: Religion, Identity and Cultural ...",None,['Edward Long'],None,http://books.google.nl/books?id=399SPgAACAAJ&d...,None,2003-03-01,http://books.google.nl/books?id=399SPgAACAAJ&d...,None,NaN,5.000000,['0253338352']


In [6]:
books_data = books_data.fillna('')

In [7]:
model = "intfloat/multilingual-e5-large"

In [8]:
embeddings = HuggingFaceEmbeddings(model_name=model)

vector_store = Chroma(
    collection_name="books",
    embedding_function=embeddings,
    persist_directory="/home/karysoares/Documents/book-reviews/notebooks"
)

print("Loaded vector store")
documents = [
    Document(page_content=row['description'], metadata=row.to_dict())
    for n, row in books_data.iterrows()
    if isinstance(row['description'], str)
]
documents.extend([
    Document(page_content=row['Title'], metadata=row.to_dict())
    for n, row in books_data.iterrows()
    if isinstance(row['Title'], str)
])
print("Finished with formatting docs")

# Vamos rodar em chuncks para facilitar a visualização do progresso

chunk_size = 100

for i in range(0, len(documents), chunk_size):
    print(f"Adding chunk {i} to vector store")
    vector_store.add_documents(documents[i:i+chunk_size])

/home/karysoares/Documents/book-reviews/.venv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/160k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/690 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

Loaded vector store
Finished with formatting docs
Adding chunk 0 to vector store


KeyboardInterrupt: 